In [1]:
import sys
import sklearn
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

import tensorflow as tf
from tensorflow import keras


# Load California Housing Dataset

Let's start by loading the California housing dataset from sklearn.datasets

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [3]:
X_train_full.shape

(15480, 8)

Scale the train, validation and test data using standard scaler

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

**Fine Tune your model: Hyperparameter optimization using Grid Search HPO**

In [5]:
!pip install scikeras

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.2
    Uninstalling importlib-metadata-4.11.2:
      Successfully uninstalled importlib-metadata-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


In [6]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from scikeras.wrappers import KerasClassifier, KerasRegressor

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
  # create model
  model = Sequential()
  model.add(Dense(30, activation="relu", input_shape=[8]))
  model.add(Dense(30, activation="relu"))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='linear'))
  # Compile model
  
  model.compile(loss='mse',optimizer='adam',metrics=['mse']) 
  return model

In [7]:
# create model
model = KerasRegressor(build_fn=create_model, epochs=50, batch_size=32, verbose=0)

In [11]:
from numpy import random
from scipy.stats import reciprocal
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV #Grid Search HPO


# define the grid search parameters
#batch_size = [32,64,128]
epochs = [50, 70]
optimizer = ['adam','rmsprop']
param_grid = dict(optimizer=optimizer,epochs=epochs)
#param_grid = dict(optimizer=optimizer,batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
grid_result = grid.fit(X_train, y_train)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"


In [12]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.779539 using {'epochs': 70, 'optimizer': 'adam'}
0.776430 (0.005589) with: {'epochs': 50, 'optimizer': 'adam'}
0.779215 (0.002688) with: {'epochs': 50, 'optimizer': 'rmsprop'}
0.779539 (0.008717) with: {'epochs': 70, 'optimizer': 'adam'}
0.777716 (0.006859) with: {'epochs': 70, 'optimizer': 'rmsprop'}


#Exercise:

Add the following parameters for Grid search CV. Run it again and observe the results.

batch_size = [32,64,128]

epochs = [50, 70, 90, 100]